In [10]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import lxml

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd

# browser = webdriver.Safari()
# browser.get('https://trudvsem.ru/vacancy/search')
# time.sleep(10)
# browser.quit()

**ISSUES:**
- TBD

**NEW_FEATURES**
- Скроллинг и клик на каждую вакансию (DONE!)
- Написан класс для парсинга сайта
- Учтена логика сброса фильтров

In [4]:
df = pd.DataFrame(columns=[
    'название',
    'компания',
    'регион',
    'зп',
    'дата обновы',
    'ссылка'
])

In [77]:
class TrudVsemParser:
    '''
    Класс для парсинга вакансий с сайта TrudVsem
    '''
    url = 'https://trudvsem.ru/vacancy/search'
    
    # 
    def __init__(self, vac_type):
        '''
        Для инциалиазации задается название вакансии vac_type
        '''
        self.vac_type = vac_type
        self.browser = webdriver.Chrome()
        self.browser.get(self.url)
        self.browser.maximize_window()
        self.browser.delete_all_cookies()
        time.sleep(4)
        
    def stop(self):
        '''
        Выйти из Webdriver selenium
        '''
        self.browser.quit()
    
    def find_vacancies(self):
        '''
        Функция для ввода названия вакансии (vac_type) 
        в поисковую строку и клика клавиши Enter
        
        После запроса убирает фильтры
        '''
        inp = self.browser.find_element(By.XPATH, '/html/body/header/div[1]/div[1]/label[1]/input')
        inp.send_keys(self.vac_type)
        inp.send_keys(Keys.ENTER)
        time.sleep(2)
        
        # убираем фильтры
        self.browser.find_element(By.XPATH, '/html/body/header/div[2]/div/div[3]/label/button').click()
        self.browser.find_element(By.XPATH, '/html/body/header/div[2]/div/div[3]/div/button').click()
        self.browser.find_element(By.XPATH, '/html/body/header/div[2]/div/div[3]/label/button').click()
        time.sleep(2)
        
    def find_first_vac(self):
        '''
        Поиск элемента первой вакансии
        '''
        self.browser.find_element(By.XPATH, '/html/body/main/div[2]/div[5]/div/div[1]/div[3]/div[1]').click()
        first_vac = self.browser.find_element(By.XPATH, '/html/body/main/div[2]/div[5]/div/div[1]/div[3]/div[1]')
        self.browser.execute_script("return arguments[0].scrollIntoView(true);", first_vac)
        time.sleep(1)
        print(first_vac.text)
        print(self.get_info_from_vac())
        print('-'*100)
        self.vac = first_vac
        
    def find_next_vac(self, vac):
        '''
        Поиск следующего элемента вакансии от вакансии, заданной в фукцнии (vac)
        После выполнения функции переприсваивает переменную vac, как активный элемент на сайте
        '''
        next_vac = vac.find_element(By.XPATH, "following-sibling::*[1]")
        self.browser.execute_script("return arguments[0].scrollIntoView(true);", next_vac)
        time.sleep(1)
        print(next_vac.text)
        print(self.get_info_from_vac())
        print('-'*100)
        next_vac.click()
        next_vac = self.browser.find_element(By.CLASS_NAME, 'search-results-simple-card_active')
        self.vac = next_vac
        time.sleep(1)
        
    def start_to_parse(self):
        '''
        Функция для полного парсинга вакансий с сайта
        
        Кликает на каждый объект и вытягивает полную информацию с блока вакансии, которая кликнута
        '''
        self.find_first_vac()
        while True:
            try:
                self.find_next_vac(self.vac)
            except:
                break
        
    def get_info_from_vac(self):
        '''
        Функция для вывода полной информации с кликнутой вакансии
        '''
        full_vac = self.browser.find_element(By.CLASS_NAME, 'search-results-full-card')
        return full_vac.text

In [78]:
parser = TrudVsemParser('системный аналитик')
parser.find_vacancies()
parser.start_to_parse()
parser.stop()

Системный аналитик
АО "МПЗ"
Владимирская область
49 000 - 63 000 руб.
Обновлено: 06.10.2023
Системный аналитик
АКЦИОНЕРНОЕ ОБЩЕСТВО "МУРОМСКИЙ ПРИБОРОСТРОИТЕЛЬНЫЙ ЗАВОД"
23.06.2023 года с 10.00 до 15.00 Всероссийская ярмарка трудоустройства. Место проведения : филиал ГКУ ВО ЦЗН Владимирской области "Муромский" по адресу: г.Муром, ул. Куйбышева 2/1
49 000 - 63 000 руб.
Откликнуться
Обновлено: 06.10.2023
Детали вакансии О компании
Должностные обязанности
Разработка, отладка, проверка работоспособности, модификация программного обеспечения. Ремонт комп.техники.
Требования к кандидату
Техническое образование в области комп.- вычислит.техники
Опыт работы от 1 года
Образование: Высшее
Данные по вакансии
Профессия: Программист
График работы: Полный рабочий день
Количество рабочих мест: 1
Контактная информация
Контактное лицо: Харитонова Ирина Васильевна
Телефон: 4923460998
Мобильный телефон: 9209406067
Email: i.v.haritonova@mpzflame.ru
Дополнительная информация по адресу: 23.06.2023 года с 10

In [35]:
with webdriver.Chrome() as browser:
    
    # заход на сайт
    browser.get('https://trudvsem.ru/vacancy/search')
    browser.maximize_window()
    browser.delete_all_cookies()
    time.sleep(2)
    
    # убрать фильтры 
    browser.find_element(By.XPATH, '/html/body/header/div[2]/div/div[2]/div/button[1]').click()
    
    # ищем поле ввода и вводим текст
    inp = browser.find_element(By.XPATH, '/html/body/header/div[1]/div[1]/label[1]/input')
    inp.send_keys('системный аналитик')
    time.sleep(1)
    
    # # убираем фильтры из серч панели
    # search_panel = browser.find_element(By.XPATH, '/html/body/header/div[2]/div')
    # buttons = search_panel.find_elements(By.CLASS_NAME, 'search-panel__item')
    # for button in buttons:
    #     button = button.find_element(By.TAG_NAME, 'button')
    #     button.click()
    
    # ищем кнопку "найти" и жмем ёё
    browser.find_element(By.XPATH, '/html/body/header/div[1]/div[1]/button').click()
    time.sleep(1)
    
    button = browser.find_element(By.XPATH, '/html/body/main/div[2]/div[5]/div/div[1]/div[3]/div[11]/button')
    # скроллинг до конца
    try: 
        while True:
            browser.execute_script("return arguments[0].scrollIntoView(true);", button)
            time.sleep(1)
            button.click()
            time.sleep(1)
    except:
        pass
    
    header = browser.find_element(By.TAG_NAME, 'h1')
    browser.execute_script("return arguments[0].scrollIntoView(true);", header)
    
    # посчитать количество вакансий на странице
    vac_block_bar = browser.find_element(By.CLASS_NAME, 'main__search-sidebar')
    vac_block = vac_block_bar.find_elements(By.TAG_NAME, 'div')
    vac_block = [div for div in vac_block if 'search-results-simple-card mb-1' in str(div.get_attribute('class'))]
    print('Количество вакансий:', len(vac_block))
    print('Парсим вакансии:\n')
    
    # вывести информацию о вакансии
    for vac in vac_block:
        vac_info = {}
        vac_info['название'] = vac.find_element(By.TAG_NAME, 'strong').text
        titles = vac.find_elements(By.CLASS_NAME, r'content_small')
        vac_info['компания'] = titles[0].text
        vac_info['регион'] = titles[1].text
        vac_info['зп'] = vac.find_element(By.CLASS_NAME, r'search-results-simple-card__salary').text
        vac_info['дата обновы'] = vac.find_element(By.CLASS_NAME, r'date__preview').text.strip('Обновлено: ')
        vac_info['ссылка'] = vac.find_element(By.TAG_NAME, 'a').get_attribute('href')
        df.loc[len(df)] = vac_info

Количество вакансий: 38
Парсим вакансии:



In [45]:
df.iloc[0, -1]

'https://trudvsem.ru/vacancy/card/1197746421620/85531d80-3767-11ee-99f9-0d972cff014f'

In [69]:
ua = UserAgent().chrome
headers = {'User-Agent': ua}

r = requests.get(df.iloc[1, -1], timeout=5, headers=headers, allow_redirects=True)

soup = BeautifulSoup(r.content, 'lxml')

In [72]:
df.head()

,название,компания,регион,зп,дата обновы,ссылка
0,Системный аналитик,"ООО ""ЦИФРОМЕД""",г. Москва,до 150 000 руб.,28.09.2023,https://trudvsem.ru/vacancy/card/1197746421620...
1,Системный аналитик,VK,г. Москва,,28.09.2023,https://trudvsem.ru/vacancy/card/7226c750-02f1...
2,Системный аналитик,"ФИЛИАЛ ООО ""АГР""",г. Москва,от 100 000 руб.,26.09.2023,https://trudvsem.ru/vacancy/card/97f0f670-8a79...
3,Системный аналитик,"ПАО ""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК""",г. Москва,до 90 000 руб.,26.09.2023,https://trudvsem.ru/vacancy/card/1027739555282...
4,Системный аналитик,ООО «МУЛЬТИБОНУС»,г. Москва,от 150 000 руб.,25.09.2023,https://trudvsem.ru/vacancy/card/a2a56960-745e...
